In [11]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch

In [2]:
file_path = "output/devset/devset_with_contexts.parquet.gzip"
with open(file_path, "rb") as f:
    dev_set = pd.read_parquet(f)

In [3]:
print(np.mean([len(item) for item in dev_set['bert_cls_context']]))

3.0


In [8]:
dev_set.loc[3, 'bert_cls_context']

array(['Et pengeinstituts kapitalandele erhvervet for puljemidler, hvor kunderne bærer risikoen, indgår ikke i opgørelsen af fradrag i kapitalgrundlaget.',
       'Priser for ydelser fra en netvirksomhed må ikke stige som følge af det vederlag, der erlægges ved overdragelse af virksomheder.',
       'Værdien af aktier, der i henhold til § 215 i lov om finansiel virksomhed overdrages til en forening, medregnes ikke ved opgørelse af foreningens skattepligtige indkomst.'],
      dtype=object)

In [ ]:
# load model and tokenizer and set device
model_name = ""  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu" # set up for mac here, change to cuda if needed
model.to(DEVICE)

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [35]:
tokenizer.encode(' Spørgsmål:')

[3686, 48, 1]